In [1]:
from tf_idf_my_func import *

In [2]:
import pandas as pd

In [3]:
dataset = load_dataset("billsum")

Found cached dataset billsum (/Users/emilygong/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
train = pd.DataFrame(dataset['train'] )
train = train.dropna()
test = pd.DataFrame(dataset['test'] )
test = test.dropna()
ca_test = pd.DataFrame(dataset['ca_test'] )
ca_test = ca_test.dropna()

# Preprocessing test dataset

In [5]:
example = test.iat[0, 0]

In [6]:
example_sentences = split_right(example)

In [7]:
example_sentences

['SECTION 1. ENVIRONMENTAL INFRASTRUCTURE.',
 '(a) Jackson County, Mississippi.--Section 219 of the Water Resources Development Act of 1992 (106 Stat. 4835; 110 Stat. 3757) is amended--',
 '(1) in subsection (c), by striking paragraph (5) and inserting the following:',
 "``(5) Jackson county, mississippi.--Provision of an alternative water supply and a project for the elimination or control of combined sewer overflows for Jackson County, Mississippi.''; and",
 "(2) in subsection (e)(1), by striking ``$10,000,000'' and inserting ``$20,000,000''.",
 "(b) Manchester, New Hampshire.--Section 219(e)(3) of the Water Resources Development Act of 1992 (106 Stat. 4835; 110 Stat. 3757) is amended by striking ``$10,000,000'' and inserting ``$20,000,000''.",
 "(c) Atlanta, Georgia.--Section 219(f)(1) of the Water Resources Development Act of 1992 (106 Stat. 4835; 113 Stat. 335) is amended by striking ``$25,000,000 for''.",
 "(d) Paterson, Passaic County, and Passaic Valley, New Jersey.--Section 21

In [8]:
for i in range(len(example_sentences)):
    example_sentences[i] = preprocessing(example_sentences[i])

In [12]:
example_sentences

[['section 1. environmental infrastructure.'],
 [' jackson county, mississippi.',
  'section 219 of the water resources development act of 1992 is amended'],
 [' in subsection , by striking paragraph and inserting the following'],
 [' jackson county, mississippi.',
  'provision of an alternative water supply and a project for the elimination or control of combined sewer overflows for jackson county, mississippi.',
  'and'],
 [' in subsection , by striking 10,000,000 and inserting 20,000,000.'],
 [' manchester, new hampshire.',
  'section 219 of the water resources development act of 1992 is amended by striking 10,000,000 and inserting 20,000,000.'],
 [' atlanta, georgia.',
  'section 219 of the water resources development act of 1992 is amended by striking 25,000,000 for.'],
 [' paterson, passaic county, and passaic valley, new jersey.',
  'section 219 of the water resources development act of 1992 is amended by striking 20,000,000 for.'],
 [' elizabeth and north hudson, new jersey.',


In [34]:
# stemmed_sentences = stemming(preprocessed)

# TF-IDF

## For each document, create a frequency matrix

In [35]:
freq_matrix = create_freq_matrix(preprocessed, stemmed_sentences)

## Term Frequency TF
TF(t) = (Number of times term t appears in a sentence) / (Total number of terms in the sentence)

In [36]:
tf_matrix = tf(freq_matrix)

## Inverse Document Frequency IDF

First, we calculate, “how many sentences contain a word”, Let’s call it Documents per words matrix.

Then, we apply the formula IDF(t) = log_e(Total number of sentences / Number of sentences with term t in it)

In [37]:
num_sent_per_word = num_sent_per_word(stemmed_sentences)

In [38]:
idf_matrix = idf(freq_matrix, num_sent_per_word, len(stemmed_sentences))

## TF-IDF

For the tf and idf values associated with each stem, multiply tf * idf

In [39]:
tf_idf_matrix = tf_idf(tf_matrix, idf_matrix)

## Score the Sentences & Find Avg Score

In [41]:
sentences_score = score_sentences(tf_idf_matrix)

In [49]:
threshold = average_sent_score(sentences_score)

# Generate Summary

In [61]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence in sentenceValue and sentenceValue[sentence] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [65]:
summary = _generate_summary(preprocessed, sentences_score, 1.3 * threshold)

In [67]:
preprocessed

['section 1. environmental infrastructure.',
 'jackson county, mississippi.',
 'section 219 of the water resources development act of 1992 is amended in subsection , by striking paragraph and inserting the following  jackson county, mississippi.',
 'provision of an alternative water supply and a project for the elimination or control of combined sewer overflows for jackson county, mississippi.',
 'and in subsection , by striking 10,000,000 and inserting 20,000,000. manchester, new hampshire.',
 'section 219 of the water resources development act of 1992 is amended by striking 10,000,000 and inserting 20,000,000. atlanta, georgia.',
 'section 219 of the water resources development act of 1992 is amended by striking 25,000,000 for.',
 'paterson, passaic county, and passaic valley, new jersey.',
 'section 219 of the water resources development act of 1992 is amended by striking 20,000,000 for.',
 'elizabeth and north hudson, new jersey.',
 'section 219 of the water resources development a

In [66]:
summary

' section 1. environmental infrastructure. sec. sec. 3. delaware river, pennsylvania and delaware. sec. 4. project reauthorizations. sec. 5. shore protection. sec. 6. comite river, louisiana. sec. 8. continuation of submission of certain reports by the secretary of the army. recommendations of inland waterways users board. list of authorized but unfunded studies. sec. authorization of appropriations. speaker of the house of representatives. vice president of the united states and president of the senate.'

# Evaluate the Summary

In [ ]:
from rouge import Rouge
rouge = Rouge()

scores = rouge.get_scores(result, text)